In [56]:
%load_ext autoreload
%autoreload 2

In [96]:
# text = <meta name="description" content="NANOGEN’s main goal is to improve the production process and enhance research activities to manufacture high quality, safe, effective and affordable products for more patients in the world.">
import requests
import pandas as pd
from bs4 import BeautifulSoup
from deep4deep.text_processing import remove_own_name

import urllib

In [88]:
website1 = 'https://nanogenpharma.com/'
website2 = 'https://netri.fr/'
website3 = 'https://feroscan.com/'
#both should work and throw an error: WHY?

In [89]:
from deep4deep.text_retrieval import get_meta_description

In [91]:
test = pd.DataFrame()
test['website_url']=[website1, website2, website3]
test['name']=['nanogen','netri', 'for sale']

In [61]:
test.loc[1]

website_url    https://netri.fr/
name                       netri
Name: 1, dtype: object

In [84]:
requests.get('https://netri.fr/', verify=False)

<Response [200]>

In [92]:
get_meta_description(test.loc[2])

so far so good
website https://feroscan.com/ request threw an error


/Users/admin/.pyenv/versions/data/envs/NLP/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'feroscan.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


' '

In [97]:
#response = requests.get(test.loc[0,'website_url'])
response = urllib.urlopen(notiurl)
html = response.read().decode(encoding="iso-8859-1")

AttributeError: module 'urllib' has no attribute 'urlopen'

In [93]:
        soup = BeautifulSoup(html.content, "html.parser")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [39]:
soup = BeautifulSoup(response.content, "html.parser")

In [40]:
#description = soup.find("meta", property="og:description")["content"]
description = soup.find('meta', attrs={'name': 'description'})

In [41]:
description = description["content"] if description else ""

In [42]:
description = remove_own_name(description, row['name'])

NameError: name 'row' is not defined

In [17]:
response = requests.get(website1)
response

NameError: name 'website1' is not defined

In [9]:
soup = BeautifulSoup(response.content, "html.parser")
#description = soup.find("meta", property="og:description")["content"]

In [10]:
description = soup.find('meta', attrs={'name': 'description'})
description

<meta content="NANOGEN’s main goal is to improve the production process and enhance research activities to manufacture high quality, safe, effective and affordable products for more patients in the world." name="description"/>

In [11]:
description = description["content"] if description else ""
description

'NANOGEN’s main goal is to improve the production process and enhance research activities to manufacture high quality, safe, effective and affordable products for more patients in the world.'

In [15]:
description = remove_own_name(description, 'nanogen')

In [16]:
description

'NANOGEN’s main goal is to improve the production process and enhance research activities to manufacture high quality, safe, effective and affordable products for more patients in the world.'